In [1]:
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath("../../ml/"))

from dev_wtp_io_utils import *

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
# nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
# ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"

sys.path.append(os.path.abspath("../../ml/"))
from apply_loocv_and_save import load_and_preprocess
from dev_utils import read_yaml_for_host
import warnings


config_data = read_yaml_for_host("sst_config.yml")



python initialized for apply_loocv_and_save
cpus available; cpus to use:
28 25
28
yaml file does not contain host-specific settings for this host. Using default settings.


In [3]:
nonbids_data_path = config_data['nonbids_data_path']
ml_data_folderpath = nonbids_data_path + "fMRI/ml"


In [4]:
nonbids_data_path

'/gpfs/projects/sanlab/shared/DEV/nonbids_data/'

In [5]:
#test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20210601T183243.csv")
#test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20211027T173724.csv")
test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20220818T144138.csv")
test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20230102T164214.csv")


In [6]:
include_exclude_list = pd.read_csv("../nsc_subject_exclusions.csv")

In [7]:
test_train_df_raw = test_train_df_raw.merge(include_exclude_list[include_exclude_list.Task=='SST'],left_on='sub_label',right_on='SubjectId',how='left')
test_train_df_raw.loc[test_train_df_raw.Include.isna(),'Include'] = True
test_train_df = test_train_df_raw[test_train_df_raw.Include==True]

In [8]:
exclude_subjects = [
    'DEV012','DEV018', 'DEV068', 'DEV073','DEV159', #excluded during the conversion process for reasons I'm not sure about

    'DEV061','DEV185','DEV187',
    'DEV189','DEV190','DEV192',
    'DEV198','DEV203','DEV220','DEV221']
#2022-01-06 added DEV157 because of an error
#2023-01-02 to -03 added DEV068, DEV101 because of an error

In [9]:
train_subjs = test_train_df.loc[test_train_df.SplitGroup_75_25=='Train','sub_label'].tolist()#only get the train subjects; ignore those previously marked hold-out

In [10]:
def get_Brain_Data_for_sub(sub_label,behavdesign,betaseries_path='/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries_nos/'):
    return(dev_wtp_io_utils.get_Brain_Data_betas_for_sub(sub_label,behavdesign,betaseries_path))

In [12]:
from dev_wtp_io_utils import import_sst_betaseries_w1_subjs_to_pkl

In [13]:
all_sst_events= pd.read_csv(ml_data_folderpath +"/SST/" + "all_sst_events.csv")
#cut out other waves
sst_1_behavdesign_1 = all_sst_events[(all_sst_events.wave=='wave1')].reset_index(drop=True)
#label the ITI as such
sst_1_behavdesign_1.loc[(all_sst_events.trial_type.isna()),'trial_type']='ITI'
#ensure it is correctly ordered
sst_1_behavdesign_clean = sst_1_behavdesign_1.sort_values(['subject','onset']).reset_index(drop=True)
#now tell it the name of the corresponding beta. this is easy; they are all consecutive
sst_1_behavdesign_clean['beta'] = 'beta_' + (sst_1_behavdesign_clean.groupby(['subject','wave']).cumcount()+1).astype(str).apply(lambda x: x.zfill(4)) + '.nii'

In [14]:
#I want to import this other module; we don't actually use it but it's a good reference
#for future programmers to see
#it's important we keep the logic consistent for the two main conditions
sys.path.append(os.path.abspath("../../fx/multiconds/SST/"))
from multiconds import create_posterror_masks_from_masks

#sst_1_behavdesign_clean['prev_trial_type'] = sst_1_behavdesign_clean['trial_type']
sst_1_behavdesign_clean['prev_trial_type'] = (
    sst_1_behavdesign_clean
    .loc[sst_1_behavdesign_clean['trial_type']!='ITI']
    .groupby(['subject','wave'])
    .shift(1)
    ['trial_type']
)#this works because during assignment, items will be assigned to the line corresponding to their index
#very important you don't reset index during this operation.
is_post_stop_correct_go = (
    (sst_1_behavdesign_clean['trial_type']=='correct-go') & 
    (sst_1_behavdesign_clean['prev_trial_type'].isin(['failed-stop','correct-stop']))
)

sst_1_behavdesign_clean.loc[is_post_stop_correct_go,'post_stop_correct_go_subtype'] = 'Other'
sst_1_behavdesign_clean.loc[is_post_stop_correct_go & (sst_1_behavdesign_clean['prev_trial_type']=='failed-stop'),'post_stop_correct_go_subtype'] = 'post-error'
sst_1_behavdesign_clean.loc[is_post_stop_correct_go & (sst_1_behavdesign_clean['prev_trial_type']=='correct-stop'),'post_stop_correct_go_subtype'] = 'post-correct'


In [15]:
sst_1_behavdesign_clean.groupby(['trial_type','post_stop_correct_go_subtype']).size().reset_index()

,trial_type,post_stop_correct_go_subtype,0
0,correct-go,post-correct,1879
1,correct-go,post-error,2145


In [16]:
sst_1_behavdesign_clean['trial_type'].value_counts()

ITI             26112
correct-go      18750
failed-stop      3434
correct-stop     3094
failed-go         834
Name: trial_type, dtype: int64

In [17]:
behavdesign = sst_1_behavdesign_clean
sst_1_behavdesign_clean_dev005 = behavdesign[[s in 'DEV005' for s in behavdesign.subject]]

## Just subjects in the conditions of interest: CorrectStop vs. CorrectGo

In [18]:
sst_1_behavdesign_clean.trial_type.value_counts()

ITI             26112
correct-go      18750
failed-stop      3434
correct-stop     3094
failed-go         834
Name: trial_type, dtype: int64

In [19]:
sst_1_behavdesign_correct_conditions=sst_1_behavdesign_clean[sst_1_behavdesign_clean.trial_type.isin(['correct-go','correct-stop'])]

In [20]:
sst_1_behavdesign_correct_conditions.trial_type.value_counts()

correct-go      18750
correct-stop     3094
Name: trial_type, dtype: int64

In [21]:
#we have data missing fro mthese subjects below. not enstirely sure yet; see 
#https://docs.google.com/document/d/1EB0ACA6qhMkDEgv-zqNTepPCe_x41fRQHaeqT47vazM/edit#heading=h.60rb13rlxkrq


train_subjs_selected = [ts for ts in train_subjs if (ts not in exclude_subjects)]

In [22]:
", ".join(test_train_df[test_train_df.SplitGroup_75_25=="Holdout"].sub_label)

'DEV007, DEV072, DEV078, DEV082, DEV090, DEV093, DEV095, DEV096, DEV098, DEV099, DEV103, DEV106, DEV108, DEV110, DEV112, DEV117, DEV120, DEV121, DEV122, DEV123, DEV127, DEV131, DEV133, DEV137, DEV147, DEV150, DEV153, DEV154, DEV163, DEV166, DEV167, DEV170, DEV172, DEV179, DEV180, DEV181, DEV182, DEV188, DEV195, DEV199, DEV200, DEV204, DEV207, DEV219, DEV225, DEV008, DEV031, DEV037, DEV081, DEV091, DEV227, DEV235, DEV241, DEV246, DEV253, DEV257, DEV258, DEV261, DEV263, DEV269, DEV270, DEV271, DEV282, DEV143, DEV149, DEV298, DEV301, DEV302, DEV307, DEV308, DEV311'

In [36]:
set(train_subjs_selected).intersection(set(test_train_df[test_train_df.SplitGroup_75_25=="Holdout"].sub_label))

set()

In [23]:
", ".join(train_subjs_selected)

'DEV011, DEV020, DEV047, DEV063, DEV064, DEV067, DEV075, DEV088, DEV094, DEV097, DEV100, DEV102, DEV105, DEV107, DEV111, DEV113, DEV114, DEV116, DEV118, DEV119, DEV135, DEV138, DEV141, DEV144, DEV145, DEV155, DEV156, DEV157, DEV158, DEV159, DEV161, DEV164, DEV168, DEV169, DEV171, DEV173, DEV174, DEV176, DEV177, DEV178, DEV183, DEV186, DEV191, DEV193, DEV194, DEV196, DEV197, DEV201, DEV202, DEV205, DEV206, DEV208, DEV209, DEV211, DEV215, DEV216, DEV217, DEV218, DEV222, DEV223, DEV224, DEV004, DEV005, DEV006, DEV009, DEV010, DEV012, DEV013, DEV014, DEV015, DEV016, DEV017, DEV018, DEV019, DEV021, DEV022, DEV023, DEV024, DEV025, DEV026, DEV027, DEV028, DEV029, DEV030, DEV033, DEV034, DEV035, DEV036, DEV038, DEV039, DEV040, DEV041, DEV042, DEV043, DEV044, DEV045, DEV046, DEV048, DEV049, DEV050, DEV051, DEV052, DEV053, DEV054, DEV055, DEV056, DEV057, DEV058, DEV059, DEV060, DEV062, DEV065, DEV066, DEV068, DEV069, DEV070, DEV071, DEV073, DEV074, DEV076, DEV077, DEV079, DEV080, DEV083, DEV084,

In [24]:
sst_1_behavdesign_correct_conditions[sst_1_behavdesign_correct_conditions.subject=='DEV157']

,onset,duration,trial_type,subject,wave,beta,prev_trial_type,post_stop_correct_go_subtype
37376,0.00000,0.87197,correct-go,DEV157,wave1,beta_0001.nii,NaN,NaN
37380,6.38890,0.08329,correct-go,DEV157,wave1,beta_0005.nii,failed-stop,post-error
37384,15.65280,0.59839,correct-go,DEV157,wave1,beta_0009.nii,failed-stop,post-error
37386,20.53475,0.46574,correct-go,DEV157,wave1,beta_0011.nii,correct-go,NaN
37388,23.35212,0.34208,correct-go,DEV157,wave1,beta_0013.nii,correct-go,NaN
...,...,...,...,...,...,...,...,...
37618,400.61111,0.43916,correct-go,DEV157,wave1,beta_0243.nii,correct-go,NaN
37620,404.11667,0.72024,correct-go,DEV157,wave1,beta_0245.nii,correct-go,NaN
37622,407.24723,0.43989,correct-go,DEV157,wave1,beta_0247.nii,correct-go,NaN
37626,412.44516,0.00170,correct-go,DEV157,wave1,beta_0251.nii,failed-stop,post-error


### Now mask those subjects

In [25]:
# get the PFC mask
mask_pfc = nib.load(ml_data_folderpath + '/prefrontal_cortex.nii.gz')

In [26]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:2],'betaseries_nos',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    out_file_suffix = '_correct_cond_pfc',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':False,
        'mask':mask_pfc
    }
    )

running
really
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries_nos/
DEV011
...............................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration  trial_type subject   wave           beta  \
1794    2.75834   0.64209  correct-go  DEV011  wave1  beta_0003.nii   
1796    5.51390   0.56936  correct-go  DEV011  wave1  beta_0005.nii   
1798    9.02224   0.46994  correct-go  DEV011  wave1  beta_0007.nii   
1800   12.52780   0.40205  correct-go  DEV011  wave1  beta_0009.nii   
1802   15.90975   0.44627  correct-go  DEV011  wave1  beta_0011.nii   
...         ...       ...         ...     ...    ...            ...   
2

In [27]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:6],'betaseries_nos',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    out_file_suffix = '_correct_cond_pfc',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':False,
        'mask':mask_pfc
    }
)

running
really
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries_nos/
DEV011
...............................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration  trial_type subject   wave           beta  \
1794    2.75834   0.64209  correct-go  DEV011  wave1  beta_0003.nii   
1796    5.51390   0.56936  correct-go  DEV011  wave1  beta_0005.nii   
1798    9.02224   0.46994  correct-go  DEV011  wave1  beta_0007.nii   
1800   12.52780   0.40205  correct-go  DEV011  wave1  beta_0009.nii   
1802   15.90975   0.44627  correct-go  DEV011  wave1  beta_0011.nii   
...         ...       ...         ...     ...    ...            ...   
2

converting to nifti and fdata
           onset  duration    trial_type subject   wave           beta  \
15104    0.00000   0.66403    correct-go  DEV064  wave1  beta_0001.nii   
15108    6.38890   0.60191    correct-go  DEV064  wave1  beta_0005.nii   
15112   15.65280   0.50451    correct-go  DEV064  wave1  beta_0009.nii   
15114   20.53475   0.39299    correct-go  DEV064  wave1  beta_0011.nii   
15116   23.35212   0.45068    correct-go  DEV064  wave1  beta_0013.nii   
...          ...       ...           ...     ...    ...            ...   
15348  404.11667   0.52668    correct-go  DEV064  wave1  beta_0245.nii   
15350  407.24723   0.59783    correct-go  DEV064  wave1  beta_0247.nii   
15354  412.44516   0.72932    correct-go  DEV064  wave1  beta_0251.nii   
15356  416.20072   2.51110  correct-stop  DEV064  wave1  beta_0253.nii   
15358  421.21182   0.81115    correct-go  DEV064  wave1  beta_0255.nii   

      prev_trial_type post_stop_correct_go_subtype  
15104             NaN       

In [28]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:30],'betaseries_nos',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    out_file_suffix = '_correct_cond_pfc',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':True,
        'mask':mask_pfc
    }
)

running
really
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries_nos/
DEV011
...............................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
-1.9874082
35.216568
-0.025306031
1.0137758
          onset  duration  trial_type subject   wave           beta  \
1794    2.75834   0.64209  correct-go  DEV011  wave1  beta_0003.nii   
1796    5.51390   0.56936  correct-go  DEV011  wave1  beta_0005.nii   
1798    9.02224   0.46994  correct-go  DEV011  wave1  beta_0007.nii   
1800   12.52780   0.40205  correct-go  DEV011  wave1  beta_0009.nii   
1802   15.90975   0.44627  correct-go  DEV011  wave1  beta_0011.nii   
...         ...       ...   

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:574: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
1.4502941
21.833902
0.037751976
1.0412676
          onset  duration    trial_type subject   wave           beta  \
4098    2.75834   0.75769    correct-go  DEV020  wave1  beta_0003.nii   
4100    5.51390   0.53414    correct-go  DEV020  wave1  beta_0005.nii   
4102    9.02224   0.60132    correct-go  DEV020  wave1  beta_0007.nii   
4104   12.52780   0.62845    correct-go  DEV020  wave1  beta_0009.nii   
4106   15.90975   0.63436    correct-go  DEV020  wave1  beta_0011.nii   
...         ...       ...           ...     ...    ...            ...   
4340  401.49167   2.00556  correct-stop  DEV020  wave1  beta_0245.nii   
4342  403.99723   0.60572    correct-go  DEV020  wave1  beta_0247.nii   
4344  407.31460   0.57269    correct-go  DEV020  wave1  beta_0249.nii   
4346  410.82016   0.48897    correct-go  DEV020  wave1  beta_0251.

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:574: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
-4.1844344
24.49937
-0.010184274
1.0110923
           onset  duration    trial_type subject   wave           beta  \
11008    0.00000   0.96259    correct-go  DEV047  wave1  beta_0001.nii   
11012    6.38890   0.58196    correct-go  DEV047  wave1  beta_0005.nii   
11016   15.65280   0.60373    correct-go  DEV047  wave1  beta_0009.nii   
11018   20.53475   0.56612    correct-go  DEV047  wave1  beta_0011.nii   
11020   23.35212   0.63492    correct-go  DEV047  wave1  beta_0013.nii   
...          ...       ...           ...     ...    ...            ...   
11252  404.11667   0.50827    correct-go  DEV047  wave1  beta_0245.nii   
11254  407.24723   0.55187    correct-go  DEV047  wave1  beta_0247.nii   
11256  409.68960   2.00556  correct-stop  DEV047  wave1  beta_0249.nii   
11258  412.44516   0.54573    correct-go  DEV047  wave1

...

...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
-0.93094045
16.302166
-0.109055065
1.0795051
           onset  duration  trial_type subject   wave           beta  \
37634    2.75834   0.90707  correct-go  DEV158  wave1  beta_0003.nii   
37636    5.51390   0.86909  correct-go  DEV158  wave1  beta_0005.nii   
37638    9.02224   0.38633  correct-go  DEV158  wave1  beta_0007.nii   
37640   12.52780   0.47927  correct-go  DEV158  wave1  beta_0009.nii   
37642   15.90975   0.46463  correct-go  DEV158  wave1  beta_0011.nii   
...          ...       ...         ...     ...    ...            ...   
37874  398.86111   0.00117  correct-go  DEV158  wave1  beta_0243.nii   
37878  403.99723   0.00168  correct-go  DEV158  wave1  beta_0247.nii   
37880  407.31460   0.00163  correct-go  DEV158  wave1  beta_0249.nii   
37882  410.82016   0.00154  correct-go  DEV158  wave1  beta_0251.nii   
3

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:574: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
0.61642134
11.586989
-0.0014113513
1.0094408
           onset  duration    trial_type subject   wave           beta  \
37890    3.00834   2.00556  correct-stop  DEV159  wave1  beta_0003.nii   
37892    6.38890   0.66374    correct-go  DEV159  wave1  beta_0005.nii   
37896   15.65280   0.50080    correct-go  DEV159  wave1  beta_0009.nii   
37898   20.53475   0.66674    correct-go  DEV159  wave1  beta_0011.nii   
37900   23.35212   0.60491    correct-go  DEV159  wave1  beta_0013.nii   
...          ...       ...           ...     ...    ...            ...   
38132  404.11667   0.49857    correct-go  DEV159  wave1  beta_0245.nii   
38134  407.24723   0.49633    correct-go  DEV159  wave1  beta_0247.nii   
38138  412.44516   0.66841    correct-go  DEV159  wave1  beta_0251.nii   
38140  416.20072   2.51110  correct-stop  DEV159  wav

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:574: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/Brain_Data_betaseries_nos_30subs_correct_cond_pfc.pkl


### Standardize by condition

In [29]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:2],'betaseries_nos',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    out_file_suffix = '_correct_cond_pfc_condstd',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':True,
        'mask':mask_pfc
    }
    )

running
really
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries_nos/
DEV011
...............................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
-1.9874082
35.216568
-0.025306031
1.0137758
          onset  duration  trial_type subject   wave           beta  \
1794    2.75834   0.64209  correct-go  DEV011  wave1  beta_0003.nii   
1796    5.51390   0.56936  correct-go  DEV011  wave1  beta_0005.nii   
1798    9.02224   0.46994  correct-go  DEV011  wave1  beta_0007.nii   
1800   12.52780   0.40205  correct-go  DEV011  wave1  beta_0009.nii   
1802   15.90975   0.44627  correct-go  DEV011  wave1  beta_0011.nii   
...         ...       ...   

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:574: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
1.4502941
21.833902
0.037751976
1.0412676
          onset  duration    trial_type subject   wave           beta  \
4098    2.75834   0.75769    correct-go  DEV020  wave1  beta_0003.nii   
4100    5.51390   0.53414    correct-go  DEV020  wave1  beta_0005.nii   
4102    9.02224   0.60132    correct-go  DEV020  wave1  beta_0007.nii   
4104   12.52780   0.62845    correct-go  DEV020  wave1  beta_0009.nii   
4106   15.90975   0.63436    correct-go  DEV020  wave1  beta_0011.nii   
...         ...       ...           ...     ...    ...            ...   
4340  401.49167   2.00556  correct-stop  DEV020  wave1  beta_0245.nii   
4342  403.99723   0.60572    correct-go  DEV020  wave1  beta_0247.nii   
4344  407.31460   0.57269    correct-go  DEV020  wave1  beta_0249.nii   
4346  410.82016   0.48897    correct-go  DEV020  wave1  beta_0251.

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:574: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


In [30]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:30],'betaseries_nos',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    out_file_suffix = '_correct_cond_pfc_condstd',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':True,
        'mask':mask_pfc
    }
    )

running
really
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries_nos/
DEV011
...............................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
-1.9874082
35.216568
-0.025306031
1.0137758
          onset  duration  trial_type subject   wave           beta  \
1794    2.75834   0.64209  correct-go  DEV011  wave1  beta_0003.nii   
1796    5.51390   0.56936  correct-go  DEV011  wave1  beta_0005.nii   
1798    9.02224   0.46994  correct-go  DEV011  wave1  beta_0007.nii   
1800   12.52780   0.40205  correct-go  DEV011  wave1  beta_0009.nii   
1802   15.90975   0.44627  correct-go  DEV011  wave1  beta_0011.nii   
...         ...       ...   

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:574: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
1.4502941
21.833902
0.037751976
1.0412676
          onset  duration    trial_type subject   wave           beta  \
4098    2.75834   0.75769    correct-go  DEV020  wave1  beta_0003.nii   
4100    5.51390   0.53414    correct-go  DEV020  wave1  beta_0005.nii   
4102    9.02224   0.60132    correct-go  DEV020  wave1  beta_0007.nii   
4104   12.52780   0.62845    correct-go  DEV020  wave1  beta_0009.nii   
4106   15.90975   0.63436    correct-go  DEV020  wave1  beta_0011.nii   
...         ...       ...           ...     ...    ...            ...   
4340  401.49167   2.00556  correct-stop  DEV020  wave1  beta_0245.nii   
4342  403.99723   0.60572    correct-go  DEV020  wave1  beta_0247.nii   
4344  407.31460   0.57269    correct-go  DEV020  wave1  beta_0249.nii   
4346  410.82016   0.48897    correct-go  DEV020  wave1  beta_0251.

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:574: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
-4.1844344
24.49937
-0.010184274
1.0110923
           onset  duration    trial_type subject   wave           beta  \
11008    0.00000   0.96259    correct-go  DEV047  wave1  beta_0001.nii   
11012    6.38890   0.58196    correct-go  DEV047  wave1  beta_0005.nii   
11016   15.65280   0.60373    correct-go  DEV047  wave1  beta_0009.nii   
11018   20.53475   0.56612    correct-go  DEV047  wave1  beta_0011.nii   
11020   23.35212   0.63492    correct-go  DEV047  wave1  beta_0013.nii   
...          ...       ...           ...     ...    ...            ...   
11252  404.11667   0.50827    correct-go  DEV047  wave1  beta_0245.nii   
11254  407.24723   0.55187    correct-go  DEV047  wave1  beta_0247.nii   
11256  409.68960   2.00556  correct-stop  DEV047  wave1  beta_0249.nii   
11258  412.44516   0.54573    correct-go  DEV047  wave1

...

...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
-0.93094045
16.302166
-0.109055065
1.0795051
           onset  duration  trial_type subject   wave           beta  \
37634    2.75834   0.90707  correct-go  DEV158  wave1  beta_0003.nii   
37636    5.51390   0.86909  correct-go  DEV158  wave1  beta_0005.nii   
37638    9.02224   0.38633  correct-go  DEV158  wave1  beta_0007.nii   
37640   12.52780   0.47927  correct-go  DEV158  wave1  beta_0009.nii   
37642   15.90975   0.46463  correct-go  DEV158  wave1  beta_0011.nii   
...          ...       ...         ...     ...    ...            ...   
37874  398.86111   0.00117  correct-go  DEV158  wave1  beta_0243.nii   
37878  403.99723   0.00168  correct-go  DEV158  wave1  beta_0247.nii   
37880  407.31460   0.00163  correct-go  DEV158  wave1  beta_0249.nii   
37882  410.82016   0.00154  correct-go  DEV158  wave1  beta_0251.nii   
3

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:574: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
0.61642134
11.586989
-0.0014113513
1.0094408
           onset  duration    trial_type subject   wave           beta  \
37890    3.00834   2.00556  correct-stop  DEV159  wave1  beta_0003.nii   
37892    6.38890   0.66374    correct-go  DEV159  wave1  beta_0005.nii   
37896   15.65280   0.50080    correct-go  DEV159  wave1  beta_0009.nii   
37898   20.53475   0.66674    correct-go  DEV159  wave1  beta_0011.nii   
37900   23.35212   0.60491    correct-go  DEV159  wave1  beta_0013.nii   
...          ...       ...           ...     ...    ...            ...   
38132  404.11667   0.49857    correct-go  DEV159  wave1  beta_0245.nii   
38134  407.24723   0.49633    correct-go  DEV159  wave1  beta_0247.nii   
38138  412.44516   0.66841    correct-go  DEV159  wave1  beta_0251.nii   
38140  416.20072   2.51110  correct-stop  DEV159  wav

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:574: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/Brain_Data_betaseries_nos_30subs_correct_cond_pfc_condstd.pkl


In [31]:
train_subjs_selected

['DEV011',
 'DEV020',
 'DEV047',
 'DEV063',
 'DEV064',
 'DEV067',
 'DEV075',
 'DEV088',
 'DEV094',
 'DEV097',
 'DEV100',
 'DEV102',
 'DEV105',
 'DEV107',
 'DEV111',
 'DEV113',
 'DEV114',
 'DEV116',
 'DEV118',
 'DEV119',
 'DEV135',
 'DEV138',
 'DEV141',
 'DEV144',
 'DEV145',
 'DEV155',
 'DEV156',
 'DEV157',
 'DEV158',
 'DEV159',
 'DEV161',
 'DEV164',
 'DEV168',
 'DEV169',
 'DEV171',
 'DEV173',
 'DEV174',
 'DEV176',
 'DEV177',
 'DEV178',
 'DEV183',
 'DEV186',
 'DEV191',
 'DEV193',
 'DEV194',
 'DEV196',
 'DEV197',
 'DEV201',
 'DEV202',
 'DEV205',
 'DEV206',
 'DEV208',
 'DEV209',
 'DEV211',
 'DEV215',
 'DEV216',
 'DEV217',
 'DEV218',
 'DEV222',
 'DEV223',
 'DEV224',
 'DEV004',
 'DEV005',
 'DEV006',
 'DEV009',
 'DEV010',
 'DEV012',
 'DEV013',
 'DEV014',
 'DEV015',
 'DEV016',
 'DEV017',
 'DEV018',
 'DEV019',
 'DEV021',
 'DEV022',
 'DEV023',
 'DEV024',
 'DEV025',
 'DEV026',
 'DEV027',
 'DEV028',
 'DEV029',
 'DEV030',
 'DEV033',
 'DEV034',
 'DEV035',
 'DEV036',
 'DEV038',
 'DEV039',
 'DEV040',

In [32]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs_selected,'betaseries_nos',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    out_file_suffix = '_correct_cond_pfc_condstd',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/",
    subj_brain_data_args = {
        'standardize_by_condition':True,
        'mask':mask_pfc
    }
)

running
really
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries_nos/
DEV004
..................................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
-0.95202
23.91381
-0.037716586
1.2239119
         onset  duration    trial_type subject   wave           beta  \
0      0.00000   0.63061    correct-go  DEV004  wave1  beta_0001.nii   
4      6.38890   0.43276    correct-go  DEV004  wave1  beta_0005.nii   
8     15.65280   0.69477    correct-go  DEV004  wave1  beta_0009.nii   
10    20.53475   0.85534    correct-go  DEV004  wave1  beta_0011.nii   
12    23.35212   0.48997    correct-go  DEV004  wave1  beta_0013.nii   
..         ...       .

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:574: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
0.37251446
16.304153
0.05789336
1.1924019
         onset  duration    trial_type subject   wave           beta  \
256    0.00000   2.25834  correct-stop  DEV005  wave1  beta_0001.nii   
258    2.75834   0.40082    correct-go  DEV005  wave1  beta_0003.nii   
260    5.51390   0.66191    correct-go  DEV005  wave1  beta_0005.nii   
264   12.52780   0.51712    correct-go  DEV005  wave1  beta_0009.nii   
266   15.90975   0.39906    correct-go  DEV005  wave1  beta_0011.nii   
..         ...       ...           ...     ...    ...            ...   
498  398.86111   0.49379    correct-go  DEV005  wave1  beta_0243.nii   
502  403.99723   0.47630    correct-go  DEV005  wave1  beta_0247.nii   
504  407.31460   0.48262    correct-go  DEV005  wave1  beta_0249.nii   
506  410.82016   0.40118    correct-go  DEV005  wave1  beta_0251.nii   
508 

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:574: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
-2.146018
28.443329
0.03985768
1.1086361
         onset  duration    trial_type subject   wave           beta  \
520   15.65280   0.83414    correct-go  DEV006  wave1  beta_0009.nii   
524   23.35212   0.44596    correct-go  DEV006  wave1  beta_0013.nii   
526   25.98407   0.54291    correct-go  DEV006  wave1  beta_0015.nii   
528   28.92644   0.58249    correct-go  DEV006  wave1  beta_0017.nii   
530   31.43061   0.34380    correct-go  DEV006  wave1  beta_0019.nii   
..         ...       ...           ...     ...    ...            ...   
758  407.24723   0.66796    correct-go  DEV006  wave1  beta_0247.nii   
760  409.68960   2.00556  correct-stop  DEV006  wave1  beta_0249.nii   
762  412.44516   0.61129    correct-go  DEV006  wave1  beta_0251.nii   
764  416.20072   2.51110  correct-stop  DEV006  wave1  beta_0253.nii   
766  

...

...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
-13.471885
17.864641
-0.03643454
1.0251299
           onset  duration    trial_type subject   wave           beta  \
15616    0.00000   0.67768    correct-go  DEV066  wave1  beta_0001.nii   
15620    6.38890   0.48496    correct-go  DEV066  wave1  beta_0005.nii   
15624   15.65280   0.60074    correct-go  DEV066  wave1  beta_0009.nii   
15626   20.53475   0.44913    correct-go  DEV066  wave1  beta_0011.nii   
15628   23.35212   0.44006    correct-go  DEV066  wave1  beta_0013.nii   
...          ...       ...           ...     ...    ...            ...   
15860  404.11667   0.49443    correct-go  DEV066  wave1  beta_0245.nii   
15862  407.24723   0.44952    correct-go  DEV066  wave1  beta_0247.nii   
15866  412.44516   0.55319    correct-go  DEV066  wave1  beta_0251.nii   
15868  416.20072   2.51110  correct-stop  DEV066  wave1

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:574: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
-0.13488035
59.58149
0.013791005
0.9816627
           onset  duration    trial_type subject   wave           beta  \
15878   11.77224   2.00556  correct-stop  DEV067  wave1  beta_0007.nii   
15882   20.53475   0.98298    correct-go  DEV067  wave1  beta_0011.nii   
15886   25.98407   0.00262    correct-go  DEV067  wave1  beta_0015.nii   
15888   28.92644   0.81914    correct-go  DEV067  wave1  beta_0017.nii   
15890   31.43061   0.77348    correct-go  DEV067  wave1  beta_0019.nii   
...          ...       ...           ...     ...    ...            ...   
16116  404.11667   0.82027    correct-go  DEV067  wave1  beta_0245.nii   
16118  407.24723   0.97949    correct-go  DEV067  wave1  beta_0247.nii   
16120  409.68960   2.00556  correct-stop  DEV067  wave1  beta_0249.nii   
16124  416.20072   2.51110  correct-stop  DEV067  wave1

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/dev_wtp_io_utils.py:574: RuntimeWarning: invalid value encountered in true_divide
  warnings.warn(wi.message,type(wi))


Exception: beta beta_0003.nii does not exist at /gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries_nos/sub-DEV068/beta_0003.nii